In [28]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
print(sys.version)

3.10.12 (main, Mar 22 2024, 16:50:05) [GCC 11.4.0]


In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict
import matplotlib.cm as cm
import pickle
import pymatgen
import gzip
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.periodic_table import Element
from ase.visualize.plot import plot_atoms

%config InlineBackend.figure_format='retina'

In [2]:
import numpy as np
np.__version__

'1.23.5'

In [3]:
import pandas as pd

In [4]:
pd.__version__

'1.5.1'

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 7.1 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2


In [5]:
ds_pickle = open("MoS2_test_new.pkl.gz", "rb")
with gzip.open("MoS2_test_new.pkl.gz", 'rb') as f:
    ds = pickle.load(f)

In [6]:
ds.columns

Index(['E_1', 'E_1_majority', 'E_1_minority', 'atomic_numbers',
       'atomic_positions', 'atomic_symbols', 'band_gap', 'band_gap_majority',
       'band_gap_minority', 'base', 'cell', 'cif', 'composition_string',
       'defect_concentration', 'defect_nsites', 'defects', 'description',
       'descriptor_id', 'energy', 'energy_per_atom', 'fermi_level',
       'formation_energy', 'formation_energy_per_site', 'formula', 'homo',
       'homo_majority', 'homo_minority', 'lattice', 'lumo', 'lumo_majority',
       'lumo_minority', 'normalized_homo', 'normalized_homo_majority',
       'normalized_homo_minority', 'normalized_lumo',
       'normalized_lumo_majority', 'normalized_lumo_minority', 'nsites',
       'nspecies', 'ordinal_id', 'pbc', 'space_group_no',
       'structure_formation_energy', 'structure_formation_energy_per_atom',
       'total_mag'],
      dtype='object')

In [7]:
with gzip.open("MoS2_train_ls0_new.pkl.gz", 'rb') as f:
    train_ls0 = pickle.load(f)

In [8]:
train_ls0.columns

Index(['E_1', 'E_1_majority', 'E_1_minority', 'atomic_numbers',
       'atomic_positions', 'atomic_symbols', 'band_gap', 'band_gap_majority',
       'band_gap_minority', 'base', 'cell', 'cif', 'composition_string',
       'defect_concentration', 'defect_nsites', 'defects', 'description',
       'descriptor_id', 'energy', 'energy_per_atom', 'fermi_level',
       'formation_energy', 'formation_energy_per_site', 'formula', 'homo',
       'homo_majority', 'homo_minority', 'lattice', 'lumo', 'lumo_majority',
       'lumo_minority', 'normalized_homo', 'normalized_homo_majority',
       'normalized_homo_minority', 'normalized_lumo',
       'normalized_lumo_majority', 'normalized_lumo_minority', 'nsites',
       'nspecies', 'ordinal_id', 'pbc', 'space_group_no',
       'structure_formation_energy', 'structure_formation_energy_per_atom',
       'total_mag'],
      dtype='object')

In [9]:
train_ls0.iloc[0]

E_1                                                                                  NaN
E_1_majority                                                                         NaN
E_1_minority                                                                         NaN
atomic_numbers                         [42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 4...
atomic_positions                       [[1.2761262868643541e-07, 1.8419295545177048, ...
atomic_symbols                         [Mo, Mo, Mo, Mo, Mo, Mo, Mo, Mo, Mo, Mo, Mo, M...
band_gap                                                                          1.0835
band_gap_majority                                                                 1.0835
band_gap_minority                                                                 1.0835
base                                                                                MoS2
cell                                                                           [8, 8, 1]
cif                  

In [29]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

In [16]:
class GraphConverter(CrystalGraph):
    def convert(self, *args, **kwargs):
        result = super().convert(*args, **kwargs)
        result['state'] = np.array([[1., 2., 3.]], dtype=np.float32)
        return result

In [17]:
from pymatgen.core.structure import Structure

In [18]:
structures = []
targets = []
for i in range(4):
    structure_cif = train_ls0.iloc[i].cif
    structure = Structure.from_str(structure_cif, fmt="cif")
    target = train_ls0.iloc[i].structure_formation_energy_per_atom
    structures.append(structure)
    targets.append(target)

In [19]:
structure_cif = train_ls0.iloc[5].cif
new_structure = Structure.from_str(structure_cif, fmt="cif")

In [20]:
!pip install -r megnet_requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached tensorflow-2.8.4-cp310-cp310-manylinux2010_x86_64.whl (498.1 MB)
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached monty-2023.4.10-py3-none-any.whl (67 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached tensorflow_estimator-2.8.0-py2.py3-none-any.whl (462 kB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth-2.32.0-py2.

In [34]:
nfeat_bond = 10
r_cutoff = 5
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5
nfeat_global = 3
graph_converter = GraphConverter(cutoff=r_cutoff)
model = MEGNetModel(nn_strategy="MinimumDistanceNNAll", nfeat_global=nfeat_global, nfeat_bond=nfeat_bond, graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)

# Model training
# Here, `structures` is a list of pymatgen Structure objects.
# `targets` is a corresponding list of properties.
model.train(structures, targets, epochs=10)

# Predict the property of a new structure
pred_target = model.predict_structure(new_structure)

first citing: MinimumDistanceNNAll
Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 0.6285
Epoch 2/10
1/1 [==============================] - 0s 227ms/step - loss: 0.4783
Epoch 3/10
1/1 [==============================] - 0s 216ms/step - loss: 0.3386
Epoch 4/10
1/1 [==============================] - 0s 209ms/step - loss: 0.2094
Epoch 5/10
1/1 [==============================] - 0s 204ms/step - loss: 0.0972
Epoch 6/10
1/1 [==============================] - 0s 205ms/step - loss: 0.0187
Epoch 7/10
1/1 [==============================] - 0s 204ms/step - loss: 0.0055
Epoch 8/10
1/1 [==============================] - 0s 209ms/step - loss: 0.0683
Epoch 9/10
1/1 [==============================] - 0s 205ms/step - loss: 0.1007
Epoch 10/10
1/1 [==============================] - 0s 204ms/step - loss: 0.0757


In [35]:
pred_target

array([-1.049614], dtype=float32)